# **Analyzing WCA Competitors' Relative Performances in Big Cube Events**

Chanoe Andrew Park, Nate Ellis

***

## **Introduction**

TODO
- WCA?
- Big cubes?

***

## **Data collection/curation + parsing**

In [2]:
import os
import requests
import shutil
import zipfile

data_dir = './WCA_data'
data_zip = f'{data_dir}.zip'

if os.path.isdir(data_dir):
    # Delete the existing data
    shutil.rmtree(data_dir)

if os.path.exists(data_zip):
    # Delete the zipped data
    os.remove(data_zip)

# Get data from WCA
url = 'https://www.worldcubeassociation.org/results/misc/WCA_export.tsv.zip'
r = requests.get(url, stream=True)

with open(data_zip, 'wb') as f:
    f.write(r.content)
            
# Extract zipped data
with zipfile.ZipFile(data_zip, 'r') as zf:
    zf.extractall(data_dir)

In [3]:
import numpy as np
import pandas as pd

df_avg = pd.read_csv(f'{data_dir}/WCA_export_RanksAverage.tsv', sep='\t', low_memory=False)

# Drop all events that aren't the big cube events
df_avg.drop(df_avg[(df_avg.eventId < '444') | (df_avg.eventId > '777')].index, inplace=True)

# Only keep competitors with averages in all big cube events
tmp = pd.merge(df_avg, df_avg, on='personId', suffixes=('_5', '_6'))
tmp.drop(tmp[(tmp.eventId_5 != '555') | (tmp.eventId_6 != '666')].index, inplace=True)
tmp = pd.merge(tmp, df_avg, on='personId')
df_grouped = tmp.drop(tmp[(tmp.eventId_5 != '555') | (tmp.eventId_6 != '666') | (tmp.eventId != '777')].index)
df_grouped.reset_index(drop=True, inplace=True)

# Drop eventId columns as not needed anymore
df_grouped.drop(['eventId_5', 'eventId_6', 'eventId'], axis = 1, inplace=True)

# Rename columns relating to 7x7
df_grouped.columns = [col if i < 9 else f'{col}_7' for (i, col) in enumerate(df_grouped.columns)]

# Fix time scale
df_grouped['best_5'] = np.float64(df_grouped['best_5'] / 100.0)
df_grouped['best_6'] = np.float64(df_grouped['best_6'] / 100.0)
df_grouped['best_7'] = np.float64(df_grouped['best_7'] / 100.0)

df_grouped

,personId,best_5,worldRank_5,continentRank_5,countryRank_5,best_6,worldRank_6,continentRank_6,countryRank_6,best_7,worldRank_7,continentRank_7,countryRank_7
0,2012PARK03,38.45,1,1,1,75.63,1,1,1,106.57,1,1,1
1,2010WEYE02,45.80,8,3,1,87.19,8,4,1,154.27,79,22,3
2,2009ZEMD01,42.09,3,1,1,81.90,5,1,1,120.63,4,1,1
3,2016KOLA02,39.79,2,1,1,80.36,3,2,1,122.46,5,2,1
4,2012PONC02,47.82,12,3,2,98.55,39,13,12,158.33,102,26,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4261,2015MEDI05,157.53,12043,950,202,298.06,5460,391,84,445.08,4465,304,58
4262,2012JIAN08,162.19,12315,4120,1577,357.26,5850,1835,732,545.27,4755,1573,694
4263,2010SPEL01,167.11,12568,3173,2266,331.04,5718,1516,1073,440.65,4442,1168,830
4264,2014GUPT37,169.67,12695,4238,406,349.61,5818,1826,188,488.91,4632,1523,130


***
## **Data management/representation**

***
## **Exploratory data analysis**

***
## **Hypothesis testing**

In [10]:
list = ['444', '555', '666', '777']
df_trimmed = pd.DataFrame()
for item in list:
    temp_df = df_avg[(df_avg.eventId.str[0:] == item)]
    temp_df = temp_df.drop(temp_df.iloc[3000:,:].index)
    temp_df.sort_values(by='personId')
    df_trimmed = pd.concat([df_trimmed,temp_df])
print(len(df_trimmed))
df_trimmed.head()

12000


,personId,eventId,best,worldRank,continentRank,countryRank
279766,2012PARK03,444,1988,1,1,1
279767,2010WEYE02,444,2146,2,1,1
279768,2009ZEMD01,444,2157,3,1,1
279769,2016KOLA02,444,2200,4,2,1
279770,2012PONC02,444,2312,5,2,2


In [12]:
for person in df_trimmed.personId.unique():
    df_person = df_trimmed.loc[df_trimmed['personId'] == person]
    if len(df_person) != 4:
        df_trimmed.drop(df_person.index, inplace=True)
print(len(df_trimmed))

6168


In [13]:
x4_df = df_trimmed.loc[df_trimmed['eventId'] == '444']
x5_df = df_trimmed.loc[df_trimmed['eventId'] == '555']
x6_df = df_trimmed.loc[df_trimmed['eventId'] == '666']
x7_df = df_trimmed.loc[df_trimmed['eventId'] == '777']

large_cube_df = pd.DataFrame()
large_cube_df['444'] = x4_df['best']
large_cube_df['555'] = x5_df['best']
print(large_cube_df)
print(x4_df)
print(x5_df)
print(len(df_trimmed))

         444  555
279766  1988  NaN
279767  2146  NaN
279768  2157  NaN
279769  2200  NaN
279770  2312  NaN
...      ...  ...
282751  4339  NaN
282752  4339  NaN
282756  4341  NaN
282760  4341  NaN
282762  4342  NaN

[1542 rows x 2 columns]
          personId eventId  best  worldRank  continentRank  countryRank
279766  2012PARK03     444  1988          1              1            1
279767  2010WEYE02     444  2146          2              1            1
279768  2009ZEMD01     444  2157          3              1            1
279769  2016KOLA02     444  2200          4              2            1
279770  2012PONC02     444  2312          5              2            2
...            ...     ...   ...        ...            ...          ...
282751  2013VILL01     444  4339       2983            690           71
282752  2019OUJI02     444  4339       2983           1076          442
282756  2014NERK01     444  4341       2991            693          510
282760  2017SORA01     444  4341       

***
## **Conclusion (Communication of insights attained)**